<h1 style="color: #00BFFF;">00 |</h1>

In [1]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import os # file managment
import warnings # warning messages management
import datetime # to play with dates

In [2]:
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

In [3]:
# 🎯 Specific functions
def explore_data(data): # sum & returns duplicates, NaN & empty spaces
    duplicate_rows = data.duplicated().sum()
    nan_values = data.isna().sum()
    empty_spaces = data.eq(' ').sum()
    import pandas as pd
    exploration = pd.DataFrame({"NaN": nan_values, "EmptySpaces": empty_spaces}) # New dataframe with the results
    print(f"There are {data.duplicated().sum()} duplicate rows. Also;")
    return exploration

def get_house_lifetime(row): # returns house lifetime, based on current year - (year renovated) or (year build)
    today = datetime.datetime.today().year
    if row['yr_renovated'] != 0:
        return today - row['yr_renovated']
    else:
        return today - row['yr_built']

<h2 style="color: #008080;">Data Extraction</h2>

In [4]:
file_path = os.path.join("C:/Users/apisi/01. IronData/01. GitHub/03. Projects/05_patern_pending/00_data", "datac.csv")
datac = pd.read_csv(file_path, index_col=0) # to deal with an error `Unnamed: 0` column
datac = datac.reset_index()

<h1 style="color: #00BFFF;">02 | Data cleaning</h1>

<h2 style="color: #008080;">Dealing with Duplicates</h2>

In [5]:
explore_data(datac) # sum & returns duplicates, NaN & empty spaces

There are 0 duplicate rows. Also;


,NaN,EmptySpaces
id,0,0
date,0,0
bedrooms,0,0
bathrooms,0,0
sqft_living,0,0
sqft_lot,0,0
floors,0,0
waterfront,0,0
view,0,0
condition,0,0


In [6]:
# Nothing to do here... moving on!

<h2 style="color: #008080;">Dealing with datetime formats</h2>

In [7]:
# Convert 'date' column to datetime format (it's an object after creating a new dataset)
datac['date'] = pd.to_datetime(datac['date'])

# Dates are complex. We will create 3 new columns for year, month and day.
datac['year'] = datac['date'].dt.year
datac['month'] = datac['date'].dt.month_name().str.slice(stop=3) # instead of giving them a number, we make them categoricals, and then we will encode them

<h2 style="color: #008080;">Encoding Categoricals</h2>

In [8]:
dummies = pd.get_dummies(datac['month'], prefix='month')
datac = pd.concat([datac, dummies], axis=1)

<h2 style="color: #008080;">Saving information about Seattle houses</h2>

In [10]:
# It'll have some sense later for the conclusions :)
seattle = datac[["lat","long", "zipcode"]]
seattle.to_csv("C:/Users/apisi/01. IronData/01. GitHub/03. Projects/05_patern_pending/00_data/seattle.csv")

<h2 style="color: #008080;">Dropping unnecessary features</h2>

In [12]:
datac = datac.drop('id', axis=1) # Well, obvious or NaN? I mean, it's NaN or never.
datac = datac.drop(['lat', 'long', 'zipcode', 'date', 'month'], axis=1)
# lat, long and zipcode we have them storage in seattle dataset.
# date has fulfilled it's purpose

<h2 style="color: #008080;">Life happens</h2>

In [13]:
duplicates = datac.duplicated()
print("Number of duplicated rows:", duplicates.sum())

Number of duplicated rows: 2


In [16]:
# For some reason (we tested for duplicates at the start, and repeat duplicates = datac.duplicated() in different steps), we got 2 duplicated rows after dropping some features.
# For now, we will just drop them
datac = datac.drop_duplicates()
duplicates = datac.duplicated()
print("Number of duplicated rows:", duplicates.sum())

Number of duplicated rows: 0


<h2 style="color: #008080;">Feature Engineering</h2>

In [17]:
# Apply the function to create a new column 'house_lifetime'
datac['house_lifetime'] = datac.apply(get_house_lifetime, axis=1) # returns house lifetime, based on current year S- (year renovated) or (year build)
datac = datac.drop(['yr_built', 'yr_renovated'], axis=1) # already in house lifetime

<h2 style="color: #008080;">Target coordinates</h2>

We will just kindly move our target to the right.

In [19]:
datac.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'price', 'year',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'house_lifetime'],
      dtype='object')

In [20]:
# Kindly moving our target to the right, as best practices
datac = datac[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'year',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'house_lifetime', 'price']]

In [21]:
# Let's see how it looks like !
datac.sample(10)

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,year,month_Apr,month_Aug,month_Dec,month_Feb,month_Jan,month_Jul,month_Jun,month_Mar,month_May,month_Nov,month_Oct,month_Sep,house_lifetime,price
8476,3,1.00,1410,18600,1.0,0,0,5,7,1410,0,1610,24941,2015,1,0,0,0,0,0,0,0,0,0,0,0,63.0,343000
8540,3,1.75,1440,9065,1.0,0,0,4,8,1440,0,1990,8812,2014,0,0,1,0,0,0,0,0,0,0,0,0,51.0,409000
15637,3,3.50,3020,4082,2.0,0,0,3,9,2080,940,2060,5102,2014,0,0,0,0,0,0,0,0,1,0,0,0,19.0,910000
728,3,1.75,1470,7650,1.0,0,0,3,7,1130,340,1590,7500,2014,0,0,0,0,0,0,0,0,0,1,0,0,40.0,328000
10896,2,1.00,1120,7440,1.0,0,0,4,7,1120,0,1170,7440,2014,0,1,0,0,0,0,0,0,0,0,0,0,84.0,335900
81,4,2.50,1900,5889,2.0,0,0,3,7,1900,0,1870,6405,2014,0,0,1,0,0,0,0,0,0,0,0,0,31.0,360000
14177,2,1.00,1210,8400,1.0,0,0,3,8,780,430,1860,6000,2014,0,0,0,0,0,0,0,0,0,1,0,0,23.0,824000
14975,4,3.25,3160,6327,2.0,0,0,3,9,2280,880,3160,5946,2015,1,0,0,0,0,0,0,0,0,0,0,0,19.0,840000
21357,4,3.25,2800,5291,2.0,0,0,3,7,2800,0,2380,5291,2014,0,1,0,0,0,0,0,0,0,0,0,0,12.0,325000
12708,5,3.50,3860,9600,1.0,0,3,4,8,2660,1200,2440,9600,2014,0,0,1,0,0,0,0,0,0,0,0,0,50.0,670000


<h2 style="color: #008080;">Moving on to --> 03_eda</h2>

In [19]:
cleaned = datac.copy()
cleaned.to_csv("C:/Users/apisi/01. IronData/01. GitHub/03. Projects/05_patern_pending/00_data/cleaned.csv")